In [1]:
from IPython.display import clear_output

In [2]:
import sys
import polars as pl
#sys.path.append('/Users/pa/Local Documents/GitHub Repositories/Spellbook') #for macos
sys.path.append(r'C:\Users\patom\OneDrive\Documents\Repo\Spellbook') # for windows
from spellbook import scraper, utils as sb_utils
import nest_asyncio, asyncio
nest_asyncio.apply()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\patom\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\patom\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\patom\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# For Windows, set SelectorEventLoop instead of ProactorEventLoop
if hasattr(asyncio, "WindowsSelectorEventLoopPolicy"):
    print("Setting WindowsSelectorEventLoopPolicy")
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

Setting WindowsSelectorEventLoopPolicy


In [8]:
keywords = [
    "travel card",
    "บัตรเดบิต",
    "โมบายแบงก์กิ้ง",
    "กองทุน",
    "ประกันชีวิต",
    "ประกันภัย",
    "บัตรกดเงินสด",
    "บัตรเครดิต",
    "สินเชื่อรถ",
    "สินเชื่อบ้าน",
    "สินเชื่อส่วนบุคคล",
    "บัญชี",
    ]

In [9]:
site_list = [
  'pantip.com',
  'x.com',
  'facebook.com',
  'tiktok.com',
  'twitter.com',
  'instagram.com',
]

results_list = []

for keyword in keywords:
  for site in site_list:
    keyword_mod = keyword.lower().replace(' ', '+')
    google_search_url = f"https://www.google.com/search?q={keyword_mod}+site:{site}+after:2024&as_epq={keyword_mod}"
    print(f'Crawling::{keyword_mod} Site::{site}\n{google_search_url}')

    results = await scraper.search.search_from_url(
      google_search_url,
      #extraction_schema={
      #  "baseSelector": "li.pt-list-item.pt-list-item__sr__no-img",
      #  "fields": [{"name": "url", "selector": "a", "type": "attribute", "attribute": "href"},],
      #},
      provider="google",
      max_results=None,
      return_schema=True,
      max_paginate=1,
    )

    # Add 'site' and 'keyword' to each result dict
    for r in results:
      r['site'] = site
      r['keyword'] = keyword
    results_list.extend(results)

    # Clears the current cell output
    clear_output(wait=True)

df = pl.DataFrame(results_list)
df = df.with_columns(
    pl.col('date_string').map_elements(lambda s: sb_utils.datetime_processing.parse_thai_date(s), return_dtype=pl.Date).alias("parsed_date"),
    pl.col('url').map_elements(lambda u: sb_utils.hashing.hash_function(u.strip(), algo='md5'), return_dtype=pl.Utf8).alias("id")
)
df.write_excel('.files/P_Mo_2025_Keywords_Search.xlsx')

Crawling::travel+card Site::pantip.com
https://www.google.com/search?q=travel+card+site:pantip.com+after:2024&as_epq=travel+card


NotImplementedError: 

In [ ]:
#results = asyncio.run(scraper.search.search_keywords_all_platforms(keywords))
#results

In [ ]:
#result = asyncio.run(scraper.crawler.crawl_pantip_topic())
#result